In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip ""

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
import torchvision
# Örnek bir transform tanımla (istediğin şekilde düzenleyebilirsin)
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# Veri setini yükle
dataset = datasets.ImageFolder(root='/content/drive/MyDrive/Colab Notebooks/Kidney-Cancer', transform=transform)
BATCH_SIZE = 16
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

train_set, test_set = torch.utils.data.random_split(dataset, [train_size, test_size])

# Eğitim verisini %10 doğrulama için ayır
val_size = int(0.1 * len(train_set))
train_size -= val_size  # Geriye kalan kısım eğitim verisi olacak

train_set, val_set = torch.utils.data.random_split(train_set, [train_size, val_size])

# DataLoader'ları oluştur (batch_size'i ihtiyacınıza göre ayarlayabilirsin)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
model = torchvision.models.efficientnet_b0(weights=weights).to(device)
model.classifier  = nn.Linear(1280,len(dataset.classes))
for param in model.parameters():
    param.requires_grad = False

for param in model.classifier.parameters():
    param.requires_grad = True

lossF = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr=0.001)


epochs = 15

for epoch in range(epochs):
    model.train().to(device)

    for inputs , labels in train_loader:
        inputs , labels = inputs.to(device),labels.to(device)
        optim.zero_grad()
        results = model(inputs)
        loss = lossF(results,labels)
        loss.backward()
        optim.step()

    print("Epoch : {}/{} , train loss : {}".format(epoch+1,epochs,loss.item()))


    model.eval()
    test_loss = 0
    acc = 0

    with torch.no_grad():
        for inputs,labels in val_loader:
            inputs , labels = inputs.to(device),labels.to(device)
            results = model(inputs)
            test_loss += lossF(results,labels).item()
            _,predicted = torch.max(results,1)
            acc += (predicted == labels).sum().item()

    test_loss  /= len(val_loader.dataset)
    accuracy  = acc/ len(val_loader.dataset)*100

    print(" Test loss : {} , test accuracy : {}".format(test_loss,accuracy))

Epoch : 1/15 , train loss : 0.15785489976406097
 Test loss : 0.005134446080774069 , test accuracy : 98.25
Epoch : 2/15 , train loss : 0.06802041828632355
 Test loss : 0.00328606691211462 , test accuracy : 98.75
Epoch : 3/15 , train loss : 0.017303630709648132
 Test loss : 0.002180649401852861 , test accuracy : 99.25
Epoch : 4/15 , train loss : 0.21519432961940765
 Test loss : 0.0020148524834075945 , test accuracy : 99.25
Epoch : 5/15 , train loss : 0.041059933602809906
 Test loss : 0.0016860045652720146 , test accuracy : 99.375
Epoch : 6/15 , train loss : 0.037676598876714706
 Test loss : 0.0014355286309728399 , test accuracy : 99.375
Epoch : 7/15 , train loss : 0.010250660590827465
 Test loss : 0.001237422492413316 , test accuracy : 99.75
Epoch : 8/15 , train loss : 0.12808501720428467
 Test loss : 0.001147534672782058 , test accuracy : 99.5
Epoch : 9/15 , train loss : 0.0682072639465332
 Test loss : 0.0016344144773029257 , test accuracy : 99.25
Epoch : 10/15 , train loss : 0.05211406

In [ ]:
torch.save(model.state_dict(), "transfer-learning-1.pth")